In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
#Getting dates and gameid for each game in 2020/2021 English Premier League

url='https://api.sportsdata.io/v3/soccer/scores/json/CompetitionDetails/EPL'

response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "9b406f3bf146465baf4d1a70bfa055ac"})

result=pd.json_normalize(response.json())
games = pd.DataFrame(result['Games'][0])#.flatten()#.columns

games_final = games[['GameId','AwayTeamId','HomeTeamId','Day','AwayTeamName','HomeTeamName']]

Days=games_final['Day'].unique().tolist()

Days=[element[0:10] for element in Days]

games_final['Day'] = pd.to_datetime(games_final['Day'])

TeamID = games_final['AwayTeamId'].unique().tolist()

Days=Days[:18]  ##### IMPORTANT: FOR DATES BEYOND THIS, EVENTS STILL HAVEN'T HAPPEN, SO CODE DOESN'T WORK

#Function that returns a dataframe of the games for each date

def daily_games(date):
    
    url='https://api.sportsdata.io/v3/soccer/odds/json/GameOddsByDate/'+date

    response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "9b406f3bf146465baf4d1a70bfa055ac"})

    response=response.json()
    result=pd.DataFrame(response)

    #Selecting games only from EPL

    result=result[result['RoundId']==602].reset_index()

    
    #Selecting games which happened

    result=result[result['Status']=='Final']

    #Aggregated Odds per game 

    odds=result['PregameOdds']

    def element(x):
        
        return pd.DataFrame(x)
            
    games_odds=pd.concat(list(map(element,list(odds))))

    games=games_odds.groupby('GameId').agg({'HomeMoneyLine':'mean','AwayMoneyLine':'mean','DrawMoneyLine':'mean'})

    #Merging with initial table

    daily_games = pd.merge(left = result,
                                 right = games,
                                 how = 'inner', 
                                 left_on = "GameId", 
                                 right_on= "GameId")


    daily_games=daily_games[['GameId', 'RoundId','Day', 
                         'AwayTeamId', 'HomeTeamId', 'AwayTeamName', 'HomeTeamName','HomeTeamScore',
                         'AwayTeamScore', 'TotalScore','HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine']]

    daily_games[['Day']]=pd.to_datetime(daily_games['Day'])
    
    return daily_games

#Applying above function to all dates contained in the list previously obtained. 
#The result is a dataframe with all games, scores and bets for the 2020/2021 EPL

final_result=pd.concat(list(map(daily_games,Days)))

#Building a column for the actual winner, based on scores presented

def winner(row):
    if row['HomeTeamScore']>row['AwayTeamScore']:
        
        return row['HomeTeamName']
    
    elif row['HomeTeamScore']<row['AwayTeamScore']:
        
        return row['AwayTeamName']
    
    elif row['HomeTeamScore']==row['AwayTeamScore']:
        
        return 'Tie'

final_result['Winner']=final_result.apply(winner,axis=1)

#Building a column for the predicted winner, based on bets presented

def moneyline(row):
    minimum=min(row['HomeMoneyLine'], row['AwayMoneyLine'], row['DrawMoneyLine'])
    if row['HomeMoneyLine']==minimum:
        return row['HomeTeamName']
    elif row['AwayMoneyLine']==minimum:
        return row['AwayTeamName']
    elif row['DrawMoneyLine']==minimum:
        return 'Tie'
    
final_result['Moneyline']=final_result.apply(moneyline,axis=1)

#Rearraging the dataframe 

final_result.reset_index(inplace=True)

final_result.drop(['index'],axis=1)

final_result=final_result[['Day','HomeTeamName','AwayTeamName','Winner',
       'Moneyline']]

In [6]:
final_result.rename(columns={'Moneyline':'Prediction'},inplace=True)

In [8]:
final_result
final_result.to_csv(r'season2021.csv', index = False)

In [184]:
#final_result['Winner_ok?'] = final_result[bool(final_result['Winner'] == final_result['Prediction'])]

In [189]:
def winner_equals(row):
    if row['Winner'] == row['Prediction']:
        return 'True'
    else:
        return 'False'

final_result['Winner was Predicted?'] = final_result.apply(winner_equals,axis=1)
final_result.head()

#final_result.groupby(['HomeTeamName']).agg({'Winner was Predicted?':'sum'})

,Day,HomeTeamName,AwayTeamName,Winner,Prediction,Winner was Predicted?
0,2020-09-12,Fulham FC,Arsenal FC,Arsenal FC,Arsenal FC,True
1,2020-09-12,Crystal Palace FC,Southampton FC,Crystal Palace FC,Southampton FC,False
2,2020-09-12,Liverpool FC,Leeds United FC,Liverpool FC,Liverpool FC,True
3,2020-09-12,West Ham United FC,Newcastle United FC,Newcastle United FC,West Ham United FC,False
4,2020-09-13,West Bromwich Albion FC,Leicester City FC,Leicester City FC,Leicester City FC,True


In [196]:
df_prediction = final_result.groupby('Winner was Predicted?').agg({'Prediction':'count'})
#df_prediction['Percentagem'] = size*100/ df_prediction['Prediction']
df_prediction

NameError: name 'pred_porc_f' is not defined

In [201]:
nbr_False = 33
nbr_True = 25
size = final_result.shape[0]

pred_porc_t = nbr_True*100/size
pred_porc_n = nbr_False*100/size
print('True: ',pred_porc_t,'False:',pred_porc_n)

True:  43.10344827586207 False: 56.89655172413793


In [183]:
#home_team = final_result.groupby(['HomeTeamName']).agg({'Winner was Predicted?':'count'})
#away_team = final_result.groupby(['AwayTeamName']).agg({'Winner was Predicted?':'count'})

#home_team.rename(columns={'HomeTeamName':'Team'},inplace=True)
#away_team.rename(columns={'AwayTeamName':'Team'},inplace=True)

#home_team.set_index('Team')
#away_team.set_index('Team')

#teams_prediction = pd.merge(left=home_team, right=away_team, how='inner',left_on='HomeTeamName', right_on='AwayTeamName')